In [1]:
import numpy as np
from astropy.io import ascii
from scipy import interpolate
from bit_tools.mission_tools import Mission
from mag_converter import *

In [2]:
magcut = 26.0
DIRECTORY = '/mnt/44868D0F868D02A2/Users/veome/Google Drive/Academics/Senior Thesis/Code/Catalog/'

In [22]:
cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)

/home/veome/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ConversionWarning: Some errors were detected !
    Line #2617 (got 10 columns instead of 11)
  """Entry point for launching an IPython kernel.


In [7]:
# Get HST F814W Band from bit_tools
hst = Mission("hst")
band = hst.get_bandpass("f814w")
f814w_band_response = band.r_lam

# Change units from nm to Angstrom
# x = band.get_wavelengths() 
x = np.arange(300, 1200, 1)
f814w = interpolate.interp1d(x*10, f814w_band_response(x), fill_value="extrapolate")

In [23]:
# Get SuperBIT Shape Band from bit_tools
bit = Mission("superbit")
r_shape = bit.get_bandpass("shape").r_lam
# r_shape = bit.get_bandpass("u").r_lam

# Change units from nm to Angstrom
# x = bit.get_bandpass("shape").get_wavelengths()
x = np.arange(300, 1200, 1)
sbit_shape = interpolate.interp1d(x*10, r_shape(x), fill_value="extrapolate")

In [24]:
# Elliptical Template
elliptical_template = np.genfromtxt('galaxy_spectra_kinney96/elliptical_template.ascii', usecols=range(2), invalid_raise=False)

In [25]:
# Starburst Template
starburst_all = [[], [], [], [], [], []]
for i in range(1, 7):
    starburst_all[i-1] = np.genfromtxt('galaxy_spectra_kinney96/starb%i_template.ascii' %(i), usecols=range(2), invalid_raise=False)

starburst_template = np.mean( np.array([starburst_all[0], starburst_all[1], starburst_all[2], starburst_all[3], starburst_all[4], starburst_all[5]]), axis=0 )

In [26]:
# Initialization
cat = cosmos[cosmos[:,2] > 0] # redshift cut
# cat = cat[cat[:,5] <= magcut] # mag cut
length = cat.shape[0]
width = cat.shape[1]

print("Original Catalog Size: %i"%cosmos.shape[0])
print("New Catalog Size: %i"%length)

Original Catalog Size: 697594
New Catalog Size: 564012


In [27]:
# Mag conversion
z = cat[:,2]
mag_obs = cat[:,5]
super_e_mag = np.zeros(length)
super_starb_mag = np.zeros(length)

for i in range(length):
    if (i%(10000) == 0):
        percentage = int((i/length)*100)
        print("Object {} of {} ({}%)".format(i, length, percentage))
        
    super_e_mag[i], super_starb_mag[i] = stmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template)    
    
print("Conversion complete!")

Object 0 of 564012 (0%)
Object 10000 of 564012 (1%)
Object 20000 of 564012 (3%)
Object 30000 of 564012 (5%)
Object 40000 of 564012 (7%)
Object 50000 of 564012 (8%)
Object 60000 of 564012 (10%)
Object 70000 of 564012 (12%)
Object 80000 of 564012 (14%)
Object 90000 of 564012 (15%)
Object 100000 of 564012 (17%)
Object 110000 of 564012 (19%)
Object 120000 of 564012 (21%)
Object 130000 of 564012 (23%)
Object 140000 of 564012 (24%)
Object 150000 of 564012 (26%)
Object 160000 of 564012 (28%)
Object 170000 of 564012 (30%)
Object 180000 of 564012 (31%)
Object 190000 of 564012 (33%)
Object 200000 of 564012 (35%)
Object 210000 of 564012 (37%)
Object 220000 of 564012 (39%)
Object 230000 of 564012 (40%)
Object 240000 of 564012 (42%)
Object 250000 of 564012 (44%)
Object 260000 of 564012 (46%)
Object 270000 of 564012 (47%)
Object 280000 of 564012 (49%)
Object 290000 of 564012 (51%)
Object 300000 of 564012 (53%)
Object 310000 of 564012 (54%)
Object 320000 of 564012 (56%)
Object 330000 of 564012 (58%)


In [28]:
all_data = np.zeros((length,width+2))
all_data[:,0:width] = cat
all_data[:,width] =  np.around(super_e_mag, decimals=5)
all_data[:,width+1] = np.around(super_starb_mag, decimals=5)

In [29]:
headers = np.array(['# RA [deg]', 'Dec [deg]', 'Redshift', 'gamma1', 'gamma2', 'Magnitude', 'Weight', 'Size', 'UncorSize', 'FWHM', 'SNR_HST', 'SuperBIT E Mag', 'SuperBIT Starburst Mag'])

In [31]:
fname = 'superbit_cosmos1361_shape.gcat_ascii'
ascii.write(all_data, fname, delimiter=' ', format='fixed_width', names=headers, overwrite=True)
print("Catalog written to {}".format(fname))

Catalog written to superbit_cosmos1361_shape.gcat_ascii


In [32]:
np.amax(super_e_mag)

28.877038400783228